In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from datetime import datetime

import re 
import pandas as pd
import numpy as np
import string
import time
import sqlite3
import warnings
import os
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

In [4]:
chromedriver_path = "./chromedriver.exe" 

service = Service(executable_path=chromedriver_path)

notebook_dir = os.getcwd() 
path = notebook_dir + "\\Data"

# Construimos la ruta al archivo SQLite
sqlite_path = os.path.join(path, 'NBA_DATABASE.sqlite')

# Conectamos SQLite usando la ruta
conn = sqlite3.connect(sqlite_path)
cursor = conn.cursor()

In [ ]:
# Variables globales

# Url para crear las direcciones
base_url = 'https://www.basketball-reference.com'

season_gamecount = 1

# Temporadas de la NBA, están como comentario porque se utilizó el mismo nombre de variable para actualizar la base de datos con la temporada actual

#precovid_seasons = ['1011', '1112', '1213', '1314', '1415', '1516', '1617', '1718', '1819']
#precovid_url_years = ['2011','2012','2013','2014', '2015', '2016', '2017', '2018', '2019']
#postcovid_seasons = ['1920', '2021', '2122', '2223', '2324','2425']
postcovid_seasons = ['2425']
#postcovid_url_years = ['2020', '2021', '2022', '2023', '2024','2025']
postcovid_url_years = ['2025']

post_covid_season_dict = {'1920': {'month_len': 8, 'final_month_gamecount': 83},
                          '2021': {'month_len': 6, 'final_month_gamecount': 140},
                          '2122': {'month_len': 7, 'final_month_gamecount': 83},
                          '2223': {'month_len': 7, 'final_month_gamecount': 72},
                          '2324': {'month_len': 7, 'final_month_gamecount': 118},
                          '2425': {'month_len': 7, 'final_month_gamecount': 118}
                         }

months_list = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
# Listas para crear las columnas de la base de datos
info_columns = ['game_id', 'season', 'date', 'away_team', 'away_score', 'home_team', 'home_score', 'result']
num_columns = ['FG', 'FGA', '3P', '3PA', 'FT', 'FTA', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', '+/-',
               'FG%', '3P%', 'FT%', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg', 'BPM']


In [ ]:
# Función para crear el game_id con la fecha y el game_count, el season_id y la fecha del partido

def create_game_info(url, season_id, season_gamecount):
    # Convertir el season_id a cadena para concatenarlo correctamente
    season_id_str = str(season_id)
    
    # Convertir el game_count a una cadena de longitud 4
    game_count = str(season_gamecount)
    while len(game_count) < 4:
        game_count = '0' + game_count
    
    # Extraer partes de la fecha de la URL
    id_string = url.strip(string.ascii_letters + string.punctuation)
    year = id_string[0:4]
    month = id_string[4:6]
    day = id_string[6:8]
    
    # Formatear la fecha
    date = year + '-' + month + '-' + day
    
    # Concatenar todos los componentes para formar el game_id
    game_id = int(season_id_str + month + day + game_count)
    
    # Convertir season_id a entero para el retorno
    season_id = int(season_id)
    
    return [game_id, season_id, date]

In [ ]:
# Función para crear la información de los equipos que han jugado un partido. Devolvemos el nombre de los equipos, los puntos y el resultado del partido como un booleano.

def create_team_info(table):
    '''
    Creación de una lista con la información de los partidos con tabla html.
    
    ---
    Inputs:
    
    table: tabla html de BeautifulSoup
    ---
    Outputs:
    
    team_info: lista con la información de los equipos en un partido.
    '''
    
    # Team_ids
    id_rows = table.findAll('th', attrs={'class':'center', 'data-stat':'team', 'scope':'row'})
    team_ids = [row.text.strip() for row in id_rows]
    
    # Final_score
    scores = table.findAll('td', attrs={'class': 'center', 'data-stat': 'T'})
    final_scores = [int(score.text.strip()) for score in scores]
    
    # Guardamos el result: 0 si el equipo visitante gana, 1 si el equipo local gana
    if final_scores[0] > final_scores[1]:
        result=0
    else:
        result=1
    
    team_info = [team_ids[0], final_scores[0], team_ids[1], final_scores[1], result]
    
    return team_info

In [ ]:
# Función para crear nuestro dataframe game_info, juntando las dos listas anteriores

def create_info_df(game_info, team_info, info_columns):
    info = game_info + team_info
    info_df = pd.DataFrame([info], columns=info_columns)
    return info_df

In [ ]:
# Función para crear los dataframes de las estadísticas de los equipos y los jugadores por partido

def create_boxscores(table, game_id):

    # Ignoramos la primera fila ya que se trata del titulo de la tabla
    rows = table.findAll('tr')[1:]
    # El primer 'th' es Starters, pero se cambiará por el nombre del jugador
    headers = rows[0].findAll('th')
    # Extraemos el nombre de las columnas
    headerlist = [h.text.strip() for h in headers]
    
    # Ignoramos la primera fila (encabezados)
    data = rows[1:]
    # Extraemos los nombres de las columans
    player_names = [row.find('th').text.strip() for row in rows]
    # Extraemos las estadísticas de los jugadores
    player_stats = [[stat.text.strip() for stat in row.findAll('td')] for row in data]
    # Añadimos el nombre del jugador a las estadísticas
    for i in range(len(player_stats)):
        # ignore header with i+1
        player_stats[i].insert(0, player_names[i+1])
    
    # Creamos el dataframe de las estadísticas de los jugadores
    player_box_df = pd.DataFrame(player_stats, columns=headerlist)
    # Quitamos la fila que pone 'Reserves'
    player_box_df.drop(player_box_df[player_box_df['Starters'] == 'Reserves'].index, inplace=True)
    
    # Añadimos el game_id a la primera columna para identificar el partido
    player_box_df.insert(loc=0, column='game_id', value=game_id)
    
    # Creamos el dataframe de las estadísticas de los equipos con la última fila de la tabla que es los totales
    team_box_df = pd.DataFrame(player_box_df.iloc[-1]).T
    
    # Eliminamos la última fila de player_box_df que es la suma de los totales
    player_box_df = player_box_df[:-1].rename(columns={'Starters': 'player'})

    return player_box_df, team_box_df

In [ ]:
# Función para crear los dataframes definitivos de las estadísticas de los equipos y los jugadores
# Juntamos las estadísticas básicas y avanzadas en un solo dataframe

def merge_boxscores(boxscore_list, team_ids, scope):

    # Creamos una lista de tuplas con las estadísticas básicas y avanzadas de cada equipo/jugador
    pairs = [((boxscore_list[i]), (boxscore_list[i + 1])) for i in range(0, len(boxscore_list), 2)]
    
    clean_boxscores= []
    
    for pair in pairs:
        
        # Combinamos boxscores básicos y avanzados
        df = pd.concat([*pair], axis=1)
        # Dropeamos columnas duplicadas
        df = df.loc[:,~df.columns.duplicated()].copy()
        
        clean_boxscores.append(df)
    
    for i in range(len(clean_boxscores)):
        
        if scope=='team':
            clean_boxscores[i].rename(columns={'Starters': 'team'}, inplace=True)
            clean_boxscores[i]['team'] = team_ids[i]
            
        elif scope=='player':
            clean_boxscores[i].insert(loc=2, column='team', value=team_ids[i])
    
    return clean_boxscores

In [ ]:
# Función para manejar los valores nulos de la extracción de datos

def change_dtypes(df, num_columns):

    df.replace(to_replace='', value='-99', inplace=True)
    
    for column in num_columns:
        df[column] = df[column].astype('float64')
        
    df.replace(to_replace=-99, value=np.nan, inplace=True)
    
    return df

In [ ]:
#Función para crear la estadística PIE (Player Impact Estimate) de los jugadores

def create_PIE(player_boxes, totals):
    
    PIE_denom = (totals['PTS'] + totals['FG'] + totals['FT'] - totals['FGA'] - totals['FTA'] + totals['DRB'] + (0.5*totals['ORB']) + totals['AST'] + totals['STL'] + (0.5*totals['BLK']) - totals['PF'] - totals['TOV'])
    player_boxes['PIE'] = round((100 * (player_boxes['PTS'] + player_boxes['FG'] + player_boxes['FT'] - player_boxes['FGA'] - player_boxes['FTA'] + player_boxes['DRB'] + (0.5*player_boxes['ORB']) + player_boxes['AST'] + player_boxes['STL'] + (0.5*player_boxes['BLK']) - player_boxes['PF'] - player_boxes['TOV']) / PIE_denom), 1)
    
    return player_boxes

In [ ]:
# Función para actualizar con los partidos no registrados de la temporada actual

def get_last_season_and_month(conn):
    """
    Obtiene la última temporada y el mes del último partido registrado en la tabla game_info.
    """
    try:
        # SQL para obtener la última temporada y mes
        query = """
        SELECT season, strftime('%m', MAX(date)) AS last_month, game_id
        FROM game_info
        GROUP BY season
        ORDER BY season DESC
        LIMIT 1;
        """
        # Ejecutar la consulta
        cursor = conn.cursor()
        cursor.execute(query)
        result = cursor.fetchone()
        # Si hay resultados, retornar temporada y mes
        if result:
            return result[0], result[1], result[2]
        else:
            # No hay registros
            return None, None, None
    except Exception as e:
        print(f"Error al obtener la última temporada, mes y game_id: {e}")
        return None, None, None

In [ ]:
query = """
    SELECT * 
    FROM player_stats;
"""
query2 = """
    SELECT name FROM sqlite_master WHERE type = 'table'
"""
games_info = pd.read_sql(query, con=conn)
games_info

,game_id,player,team,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,TS%,eFG%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,BPM,PIE,date
0,192010220001,Jrue Holiday,NOP,41:05,6.0,15.0,0.400,1.0,6.0,0.167,0.0,2.0,0.0,2.0,2.0,4.0,6.0,0.0,2.0,5.0,2.0,13.0,5.9,-14.0,0.409,0.433,0.400,0.133,4.5,4.9,4.7,22.0,0.0,4.1,23.9,20.8,81.0,120.0,-6.9,2.6,2019-10-22
1,192010220001,Brandon Ingram,NOP,35:06,8.0,19.0,0.421,2.0,5.0,0.400,4.0,4.0,1.0,0.0,5.0,5.0,5.0,1.0,2.0,2.0,4.0,22.0,15.7,-19.0,0.530,0.474,0.263,0.211,0.0,14.2,6.9,24.4,1.3,4.8,8.8,26.5,111.0,113.0,3.5,8.8,2019-10-22
2,192010220001,JJ Redick,NOP,27:03,6.0,9.0,0.667,4.0,6.0,0.667,0.0,0.0,NaN,0.0,2.0,2.0,1.0,0.0,0.0,3.0,3.0,16.0,9.2,-14.0,0.889,0.889,0.667,0.000,0.0,7.4,3.6,6.3,0.0,0.0,25.0,18.1,112.0,121.0,1.0,5.2,2019-10-22
3,192010220001,Lonzo Ball,NOP,24:50,2.0,7.0,0.286,2.0,3.0,0.667,2.0,2.0,1.0,0.0,5.0,5.0,5.0,0.0,0.0,1.0,2.0,8.0,7.1,-7.0,0.508,0.429,0.429,0.286,0.0,20.1,9.7,27.6,0.0,0.0,11.3,14.6,119.0,116.0,-0.9,5.2,2019-10-22
4,192010220001,Derrick Favors,NOP,20:46,3.0,6.0,0.500,0.0,0.0,NaN,0.0,0.0,NaN,1.0,6.0,7.0,2.0,0.0,1.0,1.0,5.0,6.0,4.6,-12.0,0.500,0.500,0.000,0.000,4.5,28.9,16.2,14.4,0.0,4.1,14.3,13.8,104.0,110.0,-3.4,3.1,2019-10-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
472349,242501190625,Doug McDermott,SAC,3:23,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,106.0,-7.7,0.0,None
472350,242501190625,Jordan McLaughlin,SAC,3:23,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.7,2.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,30.8,0.0,0.0,NaN,0.0,226.0,106.0,-0.5,0.5,None
472351,242501190625,Isaiah Crawford,SAC,2:41,1.0,2.0,0.500,0.0,0.0,NaN,0.0,0.0,NaN,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,1.6,4.0,0.500,0.500,0.000,0.000,0.0,70.1,36.9,0.0,0.0,0.0,0.0,30.4,96.0,83.0,-2.9,1.5,None
472352,242501190625,Jae Crowder,SAC,Did Not Play,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


In [ ]:
# Celda para eliminar registros de partidos. Control de errores durante la extracción de datos

# # Paso 1: Filtrar los game_id de la tabla game_info donde season no sea 1920 ni 2021
# cursor.execute("""
#     SELECT game_id, season, date, away_team, away_score, home_team, home_score, result 
#     FROM game_info
#     WHERE game_id IN (202510230488);
# """)
# game_ids_a_borrar = cursor.fetchall()

# # Si hay game_ids a borrar, proceder con las eliminaciones
# if game_ids_a_borrar:
#     # Convertir la lista de tuplas a una lista simple de game_ids
#     game_ids = [game_id[0] for game_id in game_ids_a_borrar]

#     # Paso 2: Eliminar los registros de team_stats con estos game_ids
#     cursor.execute("""
#         DELETE FROM team_stats 
#         WHERE game_id IN ({});""".format(','.join('?' * len(game_ids))), game_ids)

#     # Paso 3: Eliminar los registros de player_stats con estos game_ids
#     cursor.execute("""
#         DELETE FROM player_stats 
#         WHERE game_id IN ({});""".format(','.join('?' * len(game_ids))), game_ids)

#     # Paso 4: Eliminar las filas de game_info con estos game_ids
#     cursor.execute("""
#         DELETE FROM game_info 
#         WHERE game_id IN ({});""".format(','.join('?' * len(game_ids))), game_ids)

#     # Guardar los cambios
#     conn.commit()

#     print(f"Se han eliminado {len(game_ids)} registros de game_info, team_stats y player_stats.")
# else:
#     print("No hay filas que cumplir con los criterios de eliminación.")


Se han eliminado 1 registros de game_info, team_stats y player_stats.


In [ ]:
# Extraemos los partidos para las temporadas post-covid

for i in range(len(postcovid_seasons)):
    season_id = postcovid_seasons[i]
    season_gamecount = 1
    start_url = 'https://www.basketball-reference.com/leagues/NBA_' + postcovid_url_years[i] + '_games.html'
    
    # Abrimos el navegador con la página principal de la temporada
    driver = webdriver.Chrome(service=service)
    driver.get(start_url)
    time.sleep(5) # Delay para asegurarnos de que la página se cargue completamente
    src = driver.page_source
    parser = BeautifulSoup(src, 'lxml')
    
    # Extraemos los enlaces de los meses de la temporada
    months = parser.find('div', attrs={'class': 'filter'})
    links = months.findAll('a')
    month_links = [base_url + link['href'] for link in links]
    month_links = month_links[0:post_covid_season_dict[season_id]['month_len']]
    
    for month_url in month_links:
        # No cerramos el driver, abrimos un nuevo driver para cada mes
        driver.get(month_url)
        time.sleep(5)
        src = driver.page_source
        parser = BeautifulSoup(src, 'lxml')
        table = parser.find('div', attrs={'class': 'table_container is_setup'})

        if month_url != month_links[-1]: # Comprobamos si estamos en el último mes de la temporada
            game_partial_urls = table.findAll('td', attrs={'class': 'center', 'data-stat': 'box_score_text'})
        else: # Si estamos en el último mes, filtramos los partidos de playoffs, buscando la fila 'Play-In Game' ya que la contamos para Playoffs y fue establecido post-covid
            play_in = table.find('td', string='Play-In Game').find_parent()
            play_in_row = int(play_in['data-row'])
            body = table.find('tbody')
            all_rows = body.findAll('tr', limit=play_in_row)
            
            game_rows = []
            for row in all_rows:
                try:
                    row['class']
                except KeyError:
                    game_rows.append(row)
            game_partial_urls = [row.find(attrs={'class': 'center', 'data-stat': 'box_score_text'}) for row in game_rows]
        
        game_urls = [base_url + url.a['href'] for url in game_partial_urls]

        for i in range(len(game_urls)): # Iteramos sobre los enlaces de los partidos
            driver.get(game_urls[i])
            time.sleep(5)
            src = driver.page_source
            parser = BeautifulSoup(src, 'lxml')
            
            # Operaciones para crear la tabla game_info
            id_table = parser.find('table', attrs={'class': 'suppress_all stats_table', 'id': 'line_score'}) # Tabla con ids de los equipos y resultado
            game_info = create_game_info(url=game_urls[i],
                                         season_id=season_id,
                                         season_gamecount=season_gamecount) # Creación de la lista con información del partido
            game_id = game_info[0] # game_id del partido de nuestra lista
            team_info = create_team_info(id_table) # Creación de la lista con información de los equipos
            team_ids = [team_info[0], team_info[2]] # Nombres de los equipos en nuestra lista
            info_df = create_info_df(game_info=game_info,
                                     team_info=team_info,
                                     info_columns=info_columns) # Creación del dataframe con la información del partido
            info_df.to_sql('game_info', con=conn, if_exists='append', index=False) # Guardamos la información del partido en la base de datos

            stat_tables = parser.findAll('table', attrs={'class': 'sortable stats_table now_sortable'}) # 4 tablas con las estadísticas: home basic, home advanced, away basic, away advanced
            player_box_list = [None, None, None, None]
            team_box_list = [None, None, None, None]

            for i in range(len(stat_tables)):
                player_box_list[i], team_box_list[i] = create_boxscores(stat_tables[i], game_id=game_id) # Creación de los dataframes de las estadísticas de los equipos y jugadores
            
            # Operaciones para crear la tabla team_stats
            away_team_box, home_team_box = merge_boxscores(team_box_list, team_ids=team_ids, scope='team') # Creamos boxscores de los equipos, local y visitante
            team_boxes = pd.concat([away_team_box, home_team_box])
            team_boxes.reset_index(drop=True, inplace=True)
            team_boxes = change_dtypes(team_boxes, num_columns) # Cambiamos los tipos de datos de las columnas numéricas que ya teníamos guardadas
            team_boxes.to_sql('team_stats', con=conn, if_exists='append', index=False)
            
            # Operaciones para crear la tabla player_stats
            away_player_box, home_player_box = merge_boxscores(player_box_list, team_ids=team_ids, scope='player') # Creamos boxscores de los jugadores, local y visitante
            player_boxes = pd.concat([away_player_box, home_player_box])
            player_boxes.reset_index(drop=True, inplace=True)
            player_boxes = change_dtypes(player_boxes, num_columns) # Cambiamos los tipos de datos de las columnas numéricas que ya teníamos guardadas
            totals = dict(team_boxes.loc[:,'FG':'PTS'].sum()) # Sumamos las estadísticas de los equipos para calcular el PIE
            player_boxes = create_PIE(player_boxes, totals) # Creamos la columna PIE
            player_boxes.to_sql('player_stats', con=conn, if_exists='append', index=False)

            season_gamecount += 1

# Close the driver once all operations are done
driver.quit()


In [ ]:
# Extraemos los partidos para las temporadas pre-covid

for i in range(len(precovid_seasons)):
    
    season_id = precovid_seasons[i]
    season_gamecount = 1
    start_url = 'https://www.basketball-reference.com/leagues/NBA_' + precovid_url_years[i] + '_games.html'
    
    # Abrimos el navegador con la página principal de la temporada
    driver = webdriver.Chrome(service=service)
    driver.get(start_url)
    time.sleep(5) # Delay para asegurarnos de que la página se cargue completamente
    src = driver.page_source
    parser = BeautifulSoup(src, 'lxml')
    
    # Extraemos los enlaces de los meses de la temporada
    months = parser.find('div', attrs = {'class': 'filter'})
    # Extraemos los enlaces de los meses de la temporada
    links = months.findAll('a')
    # Urls de los meses de la temporada
    month_links = [base_url + link['href'] for link in links]
    # Limitamos la extracción a los primeros 7 meses de la temporada (octubre a abril)
    month_links = month_links[0:7]
    
    for month_url in month_links:
        
        # No cerramos el driver, abrimos un nuevo driver para cada mes
        driver.get(month_url)
        time.sleep(5)
        src = driver.page_source
        parser = BeautifulSoup(src, 'lxml')
        table = parser.find('div', attrs = {'class': 'table_container is_setup'})
        
        # Comprobamos si estamos en el mes de playoffs, si es así, limitamos la extracción a los partidos antes de playoffs
        row_num = None
        splits = table.findAll('tr', attrs = {'class': 'thead'})
        for split in splits:
            if 'Playoffs' in split.text:
                row_num = int(split['data-row'])
                
        # Extraemos las urls parciales de los partidos, si es abril, limitamos la extracción a los partidos antes de playoffs
        if row_num == None:
            game_partial_urls = table.findAll('td', attrs = {'class': 'center', 'data-stat': 'box_score_text'})
        elif row_num != None:
            game_partial_urls = table.findAll('td', attrs = {'class': 'center', 'data-stat': 'box_score_text'}, limit=row_num)
        
        game_urls = [base_url + url.a['href'] for url in game_partial_urls]
        
        # Iteramos sobre los enlaces de los partidos
        for i in range(len(game_urls)):
    
            driver.get(game_urls[i])
            time.sleep(5)
            src = driver.page_source
            parser = BeautifulSoup(src, 'lxml')
            
            # Operaciones para crear la tabla game_info
            id_table = parser.find('table', attrs = {'class': 'suppress_all stats_table', 'id': 'line_score'})
            game_info = create_game_info(url=game_urls[i],
                                         season_id=season_id,
                                         season_gamecount=season_gamecount)
            game_id = game_info[0] # game_id del partido
            team_info = create_team_info(id_table) # Creación de la lista con información de los equipos
            team_ids = [team_info[0], team_info[2]] # Nombres de los equipos en nuestra lista
            info_df = create_info_df(game_info=game_info,
                                     team_info=team_info,
                                     info_columns=info_columns) # Creación del dataframe con la información del partido
            info_df.to_sql('game_info', con=conn, if_exists='append', index=False)
            
            stat_tables = parser.findAll('table', attrs = {'class': 'sortable stats_table now_sortable'}) # 4 tablas con las estadísticas: home basic, home advanced, away basic, away advanced
            player_box_list = [None, None, None, None]
            team_box_list = [None, None, None, None]

            for i in range(len(stat_tables)):
                player_box_list[i], team_box_list[i] = create_boxscores(stat_tables[i], game_id=game_id) # Creación de los dataframes de las estadísticas de los equipos y jugadores

            # Operaciones para crear la tabla team_stats
            away_team_box, home_team_box = merge_boxscores(team_box_list, team_ids=team_ids, scope='team') # Creamos boxscores de los equipos, local y visitante, junta las estadísticas básicas y avanzadas
            team_boxes = pd.concat([away_team_box, home_team_box])
            team_boxes.reset_index(drop=True, inplace=True)
            team_boxes = change_dtypes(team_boxes, num_columns) # Cambiamos los tipos de datos de las columnas numéricas que ya teníamos guardadas
            team_boxes.to_sql('team_stats', con=conn, if_exists='append', index=False)
            
            # Operaciones para crear la tabla player_stats
            away_player_box, home_player_box = merge_boxscores(player_box_list, team_ids=team_ids, scope='player') # Creamos boxscores de los jugadores, local y visitante, junta las estadísticas básicas y avanzadas
            player_boxes = pd.concat([away_player_box, home_player_box])
            player_boxes.reset_index(drop=True, inplace=True)
            player_boxes = change_dtypes(player_boxes, num_columns) # Cambiamos los tipos de datos de las columnas numéricas que ya teníamos guardadas
            totals = dict(team_boxes.loc[:,'FG':'PTS'].sum()) # Sumamos las estadísticas de los equipos para calcular el PIE
            player_boxes = create_PIE(player_boxes, totals) # Creamos la columna PIE
            player_boxes.to_sql('player_stats', con=conn, if_exists='append', index=False)

            # increase gamecount to create next game_id
            season_gamecount += 1

In [ ]:
# Extraemos los datos de los partidos de la temporada actual y que no están registrado en la base de datos. Este proceso se ejecutará una vez al día para actualizar la base de datos con los partidos más recientes.
# Se deja comentado, ya que se ha separado en el notebook Update-NBA-Scraper para utilizarlo en el pipeline de datos en el futuro.

# Obtener la temporada en curso
current_year = datetime.now().year
if datetime.now().month < 7:
    season_start = str(current_year - 1)[2:]  # Últimos dos dígitos del año anterior
    season_end = str(current_year)[2:]
    season_year = current_year        # Últimos dos dígitos del año actual
else:
    season_start = str(current_year)[2:]      # Últimos dos dígitos del año actual
    season_end = str(current_year + 1)[2:]
    season_year = current_year + 1    # Últimos dos dígitos del año siguiente

# Concatenar para formar la temporada
season_id = season_start + season_end

print(season_year)
# URL de la temporada en curso
start_url = f'https://www.basketball-reference.com/leagues/NBA_{season_year}_games.html'

# Obtener el último season_gamecount de la base de datos
cursor.execute('SELECT MAX(game_id) FROM game_info WHERE season = ?', (season_year,))
last_game_id = cursor.fetchone()[0]

if last_game_id:
    last_game_id_str = str(last_game_id)  # Convertir a cadena
    last_gamecount = int(last_game_id_str[-4:])  # Extraer los últimos 4 caracteres y convertir a entero
    season_gamecount = last_gamecount + 1
else:
    season_gamecount = 1

# Web scraping de la temporada en curso
driver = webdriver.Chrome(service=service)
driver.get(start_url)
time.sleep(5)
src = driver.page_source
parser = BeautifulSoup(src, 'lxml')

# Obtener enlaces de cada mes de la temporada
months = parser.find('div', attrs={'class': 'filter'})
links = months.findAll('a')
month_links = [base_url + link['href'] for link in links]

for month_url in month_links:
    driver.get(month_url)
    time.sleep(5)
    src = driver.page_source
    parser = BeautifulSoup(src, 'lxml')
    table = parser.find('div', attrs={'class': 'table_container is_setup'})

    game_partial_urls = table.findAll('td', attrs={'class': 'center', 'data-stat': 'box_score_text'})
    game_urls = [base_url + url.a['href'] for url in game_partial_urls if url.a]

    print(game_urls)

    for game_url in game_urls:
        # Extraer la fecha y el equipo local de la URL del partido
        match = re.search(r'boxscores/(\d{4})(\d{2})(\d{2})0([A-Z]{3})', game_url)
        if match:
            year, month, day, home_team = match.groups()
            game_date = f"{year}-{month}-{day}"
            game_date_obj = datetime.strptime(game_date, '%Y-%m-%d')

            print(match)
            print(game_date, home_team)
            print(game_date_obj)
            
            # Comprobar si el partido ya está en la base de datos
            cursor.execute('''
                SELECT COUNT(1) 
                FROM game_info 
                WHERE date = ? AND home_team = ?
            ''', (game_date, home_team))

            match_result = cursor.fetchone()
            print(match_result)

            if match_result and match_result[0] > 0:
                print(f"Game already exists: {game_date} - {home_team}, skipping...")
                continue  # Saltar si el partido ya está registrado

            # Visitar la página del partido
            driver.get(game_url)
            time.sleep(5)
            src = driver.page_source
            parser = BeautifulSoup(src, 'lxml')

            # Extraer y guardar la información del partido
            id_table = parser.find('table', attrs={'class': 'suppress_all stats_table', 'id': 'line_score'})
            print(game_url, season_year, season_gamecount)
            game_info = create_game_info(url=game_url, 
                                         season_id=season_year, 
                                         season_gamecount=season_gamecount)
            game_id = game_info[0]
            team_info = create_team_info(id_table)
            team_ids = [team_info[0], team_info[2]]
            info_df = create_info_df(game_info=game_info, 
                                     team_info=team_info, 
                                     info_columns=info_columns)
            info_df.to_sql('game_info', con=conn, if_exists='append', index=False)

            stat_tables = parser.findAll('table', attrs={'class': 'sortable stats_table now_sortable'})
            player_box_list = [None, None, None, None]
            team_box_list = [None, None, None, None]

            for j in range(len(stat_tables)):
                player_box_list[j], team_box_list[j] = create_boxscores(stat_tables[j], game_id=game_id)

            # Procesar estadísticas de equipos
            away_team_box, home_team_box = merge_boxscores(team_box_list, 
                                                           team_ids=team_ids, 
                                                           scope='team')
            team_boxes = pd.concat([away_team_box, home_team_box])
            team_boxes.reset_index(drop=True, inplace=True)
            team_boxes = change_dtypes(team_boxes, num_columns)
            team_boxes.to_sql('team_stats', con=conn, if_exists='append', index=False)

            # Procesar estadísticas de jugadores
            away_player_box, home_player_box = merge_boxscores(player_box_list, 
                                                               team_ids=team_ids, 
                                                               scope='player')
            player_boxes = pd.concat([away_player_box, home_player_box])
            player_boxes.reset_index(drop=True, inplace=True)
            player_boxes = change_dtypes(player_boxes, num_columns)
            totals = dict(team_boxes.loc[:, 'FG':'PTS'].sum())
            player_boxes = create_PIE(player_boxes, totals)
            player_boxes.to_sql('player_stats', con=conn, if_exists='append', index=False)

            season_gamecount += 1
driver.quit()

2025
['https://www.basketball-reference.com/boxscores/202410220BOS.html', 'https://www.basketball-reference.com/boxscores/202410220LAL.html', 'https://www.basketball-reference.com/boxscores/202410230DET.html', 'https://www.basketball-reference.com/boxscores/202410230ATL.html', 'https://www.basketball-reference.com/boxscores/202410230MIA.html', 'https://www.basketball-reference.com/boxscores/202410230PHI.html', 'https://www.basketball-reference.com/boxscores/202410230TOR.html', 'https://www.basketball-reference.com/boxscores/202410230HOU.html', 'https://www.basketball-reference.com/boxscores/202410230NOP.html', 'https://www.basketball-reference.com/boxscores/202410230UTA.html', 'https://www.basketball-reference.com/boxscores/202410230LAC.html', 'https://www.basketball-reference.com/boxscores/202410230POR.html', 'https://www.basketball-reference.com/boxscores/202410240WAS.html', 'https://www.basketball-reference.com/boxscores/202410240DAL.html', 'https://www.basketball-reference.com/boxs

In [ ]:
# Añadimos la date a la tabla player_stats y team_stats

query2 = """UPDATE player_stats
SET date = (SELECT game_info.date FROM game_info WHERE game_info.game_id = player_stats.game_id)
WHERE EXISTS (SELECT 1 FROM game_info WHERE game_info.game_id = player_stats.game_id);
"""

cursor.execute(query2)
conn.commit()
player_stats = pd.read_sql_query("SELECT * from player_stats", conn)
player_stats.head()